In [33]:
import numpy as np
import pandas as pd

from kiloword.utils import *
from kiloword.config import Config
from kiloword.visualisation import plot_features
from kiloword.dimension_reduction import *

## Download the data

In [34]:
metadata_df = load_kiloword_metadata()

In [35]:
# data_df = load_kiloword_metadata(Config().DATATXT)

In [36]:
data = load_data_from_fif()

Reading /home/viki/mne_data/MNE-kiloword-data/kword_metadata-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =    -100.00 ...     920.00 ms
        0 CTF compensation matrices available
Adding metadata with 8 columns
960 matching events found
No baseline correction applied
0 projection items activated


In [37]:
data = pd.read_csv(Config().MNE_PATH / "KWORD_ERP_LEXICAL_DECISION_DGMH2015.csv")
data.head()

,WORD#,WORD,ELEC#,ELECNAME,-100ms,-96ms,-92ms,-88ms,-84ms,-80ms,...,884ms,888ms,892ms,896ms,900ms,904ms,908ms,912ms,916ms,920ms
0,WORD1,film,ELEC1,REJ1,0.198182,0.385455,0.557576,0.694091,0.784545,0.813788,...,-3.252424,-3.196970,-3.107121,-3.004242,-2.897424,-2.774697,-2.613636,-2.405909,-2.179697,-1.965758
1,WORD1,film,ELEC2,REJ2,0.129091,0.209545,0.260758,0.268939,0.244697,0.200152,...,-1.348333,-1.282879,-1.198182,-1.118485,-1.030000,-0.900909,-0.711515,-0.473030,-0.214697,0.024848
2,WORD1,film,ELEC3,REJ3,0.054545,0.067273,0.056212,0.026818,-0.012879,-0.052424,...,-0.656667,-0.598788,-0.549848,-0.527727,-0.533939,-0.559545,-0.607121,-0.676818,-0.760909,-0.841061
3,WORD1,film,ELEC4,Fz,0.421970,0.453939,0.465606,0.468182,0.483485,0.518939,...,0.451667,0.690000,0.926364,1.147576,1.354091,1.553485,1.747121,1.927879,2.080303,2.199697
4,WORD1,film,ELEC5,Cz,0.251970,0.232879,0.194242,0.158939,0.159242,0.214091,...,1.160152,1.391364,1.627273,1.854242,2.066364,2.262273,2.437273,2.585152,2.705606,2.801364


In [38]:
list_words = pd.unique(data["WORD"])
list_electrodes = pd.unique(data["ELECNAME"])
grouped_data = data.groupby("WORD")
list_eegs = []
for word in list_words:
    da = grouped_data.get_group(word)
    da = da[~da['ELECNAME'].isin(["REJ1", "REJ2", "REJ3"])]#.drop(columns=['WORD#', 'WORD', 'ELEC#', 'ELECNAME'])
    d = da.drop(columns=['WORD#', 'WORD', 'ELEC#', 'ELECNAME']).to_numpy()
    list_eegs.append(d)

In [39]:
eegs = np.stack(list_eegs)

In [61]:
eegs.shape
print([(i, e) for i, e in enumerate(list_electrodes[3:])])

[(0, 'Fz'), (1, 'Cz'), (2, 'Pz'), (3, 'C3'), (4, 'Fp1'), (5, 'F7'), (6, 'T7'), (7, 'P7'), (8, 'O1'), (9, 'F3'), (10, 'FC5'), (11, 'CP5'), (12, 'P3'), (13, 'FC1'), (14, 'CP1'), (15, 'Fpz'), (16, 'FC2'), (17, 'CP2'), (18, 'Oz'), (19, 'F4'), (20, 'FC6'), (21, 'CP6'), (22, 'P4'), (23, 'C4'), (24, 'Fp2'), (25, 'F8'), (26, 'T8'), (27, 'P8'), (28, 'O2')]


In [41]:
list_words

array(['film', 'cent', 'shot', 'cold', 'main', 'poet', 'case', 'land',
       'idea', 'love', 'role', 'hall', 'safe', 'pope', 'suit', 'evil',
       'roof', 'wind', 'seat', 'pale', 'duty', 'inch', 'snow', 'site',
       'dome', 'golf', 'boil', 'gaze', 'bone', 'yarn', 'gift', 'pump',
       'trap', 'harm', 'beer', 'buzz', 'bait', 'clan', 'rift', 'jerk',
       'skit', 'dune', 'flop', 'drip', 'slat', 'moot', 'feud', 'coax',
       'issue', 'basis', 'peace', 'trial', 'least', 'green', 'daily',
       'scene', 'party', 'thing', 'price', 'piece', 'beach', 'unity',
       'chair', 'rifle', 'uncle', 'smoke', 'dream', 'crowd', 'theme',
       'wagon', 'empty', 'wheel', 'storm', 'actor', 'purse', 'chaos',
       'blade', 'sheer', 'bench', 'suite', 'cycle', 'shade', 'radar',
       'creek', 'tango', 'briar', 'patio', 'polka', 'jiffy', 'cameo',
       'felon', 'filth', 'druid', 'sprig', 'spout', 'snout', 'source',
       'mother', 'common', 'father', 'theory', 'effect', 'dinner',
       'growth',

In [42]:
PERP = 10
LEFT_CHANS = [5, 6, 7, 10, 11]

In [43]:
reshaped_eegs = eegs.reshape(-1, eegs.shape[1]*eegs.shape[2])
tsne_features = compute_tsne(reshaped_eegs, normalize="min_max", n_components=2, perplexity=PERP)
print("computed TSNE")
pca_features = compute_pca(reshaped_eegs, normalize="min_max", n_components=2)

computed TSNE


In [44]:

list_pos = pd.read_csv(Config().MNE_PATH / "words_and_pos.csv")
list_pos = list_pos["POS"].values

In [45]:
plot_features(tsne_features, list_words, list_pos, layout_title="tSNE features", layout_height=800, layout_width=800)

In [46]:
plot_features(pca_features, list_words, list_pos, layout_title="PCA", layout_height=800, layout_width=800)

### Visualize temporally-truncated data

In [47]:
TRUNC = 200
reshaped_eegs = eegs[:, :, TRUNC:]
reshaped_eegs = reshaped_eegs.reshape(-1, reshaped_eegs.shape[1]*reshaped_eegs.shape[2])
tsne_features = compute_tsne(reshaped_eegs, normalize="normal", n_components=2, perplexity=PERP)
print("computed TSNE")
pca_features = compute_pca(reshaped_eegs, normalize="normal", n_components=2)

computed TSNE


In [48]:
plot_features(tsne_features, list_words, list_pos, layout_title=f"tSNE on {TRUNC} truncated features", layout_height=800, layout_width=800)

In [49]:
plot_features(pca_features, list_words, list_pos, layout_title=f"PCA on {TRUNC} truncated features", layout_height=800, layout_width=800)

### Visualize on specific channels

In [68]:
TRUNC = 200
PERP = 10
LEFT_CHANS = [6]
for TRUNC in [100, 150, 200]:
    reshaped_eegs = eegs[:, LEFT_CHANS, TRUNC:]
    reshaped_eegs = reshaped_eegs.reshape(-1, reshaped_eegs.shape[1]*reshaped_eegs.shape[2])
    tsne_features = compute_tsne(reshaped_eegs, normalize="normal", n_components=2, perplexity=PERP)
    print("computed TSNE")
    pca_features = compute_pca(reshaped_eegs, normalize="normal", n_components=2)
    plot_features(tsne_features, list_words, list_pos, layout_title=f"tSNE on LEFT HEMI features", layout_height=800, layout_width=800)

computed TSNE


computed TSNE


computed TSNE


In [65]:
LEFT_CHANS = [6]
for TRUNC in [100, 150, 200]:
    reshaped_eegs = eegs[:, LEFT_CHANS, TRUNC:]
    reshaped_eegs = reshaped_eegs.reshape(-1, reshaped_eegs.shape[1]*reshaped_eegs.shape[2])
    pca_features = compute_pca(reshaped_eegs, normalize="normal", n_components=2)
    plot_features(pca_features, list_words, list_pos, layout_title=f"PCA on  LEFT HEMI features", layout_height=800, layout_width=800)